In [56]:
import sys
print(sys.version)

3.7.12 (default, Oct 13 2021, 06:53:03) 
[Clang 13.0.0 (clang-1300.0.29.3)]


In [57]:
%load_ext autoreload
%autoreload 0

import os, json
from draftfast import rules
from draftfast.optimize import run
from draftfast.orm import Player
from draftfast.csv_parse import salary_download
from draftfast.settings import OptimizerSettings, CustomRule, PlayerPoolSettings
from draftfast.lineup_constraints import LineupConstraints
import pandas as pd
import numpy as np
from scipy import stats
from spread import get_metabet_spread, get_current_rankings, get_fantasy_def_points_against
from collections import defaultdict

# https://www.pro-football-reference.com/years/2021/fantasy.htm

SALARY_FILE = './data/1_17.csv'
WEEK = 20
MIN_SALARY = 4799
WEIGHTED = True

ACTIVE_FILE = './data/active.csv'
MAX_PLAYED = WEEK - 1
print('ready')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
ready


In [58]:
# https://www.lineups.com/nfl-team-rankings
ranking_df = get_current_rankings(WEEK, 'MTI0NTQ2MDU3OQ==', True)

def get_abbr(x):
    try:
        if type(x) is not dict:
            x = eval(x)
        return x['team_abbr']
    except Exception as e:
        print(e,x)
        

print(ranking_df.shape)
ranking_df['team'] = ranking_df['nav'].apply(get_abbr)
# DEF_KEYS = ['passing_interceptions_rank'
# ranking_df['def_rank'] = ranking_df.apply(lambda row: np.avg([row[k] for k in DEF_KEYS]))
num_teams = ranking_df.shape[0]


return cached data week 20
(32, 52)


In [59]:
rankings = {x['team']: x for x in ranking_df.to_dict('records')}
# rankings
teams = set(rankings.keys())
print(teams, len(teams))
rankings['LV'] = rankings['OAK']
rankings['JAC'] = rankings['JAX']
del rankings['OAK']
rankings['PHI']

{'JAX', 'IND', 'ATL', 'OAK', 'LAC', 'CAR', 'NYJ', 'DEN', 'PHI', 'MIA', 'NO', 'BUF', 'CHI', 'SEA', 'MIN', 'GB', 'HOU', 'NE', 'WAS', 'TEN', 'CLE', 'CIN', 'PIT', 'NYG', 'ARI', 'LAR', 'BAL', 'TB', 'DET', 'SF', 'DAL', 'KC'} 32


{'Unnamed: 0': 2,
 'points_def': 385,
 'passing_completions': 409,
 'touchdowns': 47,
 'turnover_rank': 26,
 'rushing_attempts_rank': 18,
 'offensive_yards_rank': 10,
 'first_downs': 358,
 'rushing_touchdowns': 18,
 'offensive_rating_rank': 3,
 'turnover': 16,
 'red_zone_conversions_rank': 27,
 'red_zone_percentage_rank': 29,
 'passing_yards_rank': 11,
 'offensive_plays': 1076,
 'third_down_percentage_rank': 23,
 'nav': "{'team_logo_bordered': 'assets/images/nfl/logos/bordered/philadelphia-eagles-largest.svg', 'team_name': 'Eagles', 'opp_division': 'South', 'opp_logo_bordered': 'assets/images/nfl/logos/bordered/tampa-bay-buccaneers-largest.svg', 'team_rank': '2nd', 'opp_depth_chart_route': '/nfl/depth-charts/tampa-bay-buccaneers', 'team_conference': 'NFC', 'opp_league': 'NFC', 'opp_news_route': '/nfl/news/tampa-bay-buccaneers', 'opp_name_full': 'Tampa Bay Buccaneers', 'status': {}, 'opp_name': 'Buccaneers', 'matchup_route': None, 'matchup_news_route': '/nfl/news/matchups/phi-tb', 'game

In [60]:

spread_df = get_metabet_spread(WEEK)
favor_map = {}
z_map = {}
z_scores = {}

if 'OverUnder' in spread_df.columns.values:
    points = list(spread_df['OverUnder'])
    zs = stats.zscore(points)
    for i, p in enumerate(points):
        z_scores[p] = zs[i]

# https://madduxsports.com/how-to-read-nfl-odds-lines.html"
for index, row in spread_df.iterrows():
    home, away = row['HomeTeam'], row['AwayTeam']
    favor_map[home] = row['PointSpread']
    favor_map[away] = -row['PointSpread']
    if 'OverUnder' in row:
        z_map[home] = z_scores[row['OverUnder']]
        z_map[away] = z_scores[row['OverUnder']]
    

    
if 'JAX' in favor_map:
    favor_map['JAC'] = favor_map['JAX']
    z_map['JAC'] = z_map['JAX']
    
if 'LVS' in favor_map:
    favor_map['LV'] = favor_map['LVS']
    z_map['LV'] = z_map['LVS']

print('favor_map teams', len(favor_map))
print(favor_map)
z_map

return cached data week 20
favor_map teams 8
{'LAR': -3.47, 'ARI': 3.47, 'TEN': -3.5, 'CIN': 3.5, 'GB': -5.45, 'SF': 5.45, 'KC': -2.48, 'BUF': 2.48}


{'LAR': -0.29902289063720355,
 'ARI': -0.29902289063720355,
 'TEN': -0.7768349247624503,
 'CIN': -0.7768349247624503,
 'GB': -0.6300560646063292,
 'SF': -0.6300560646063292,
 'KC': 1.7059138800059852,
 'BUF': 1.7059138800059852}

In [61]:
df = pd.read_csv(SALARY_FILE, na_values= '')
df = df.fillna(df.median())
print(df.describe())

df['Name'] = df['First Name'] + " " + df['Last Name']
df['Salary/FPPG'] = df['FPPG'] / df['Salary']

all_teams = favor_map.keys()
winning_teams = [x for x in all_teams if favor_map[x] < 0]
ordered_teams = sorted(all_teams, key=lambda x: favor_map[x])
print('ordered_teams', [(x, favor_map[x]) for x in ordered_teams])
# excluded_teams = set([x for x in all_teams if favor_map[x] > -MIN_FAVORED])
# top_teams = winning_teams
team_offset = 1
half_teams = int(len(ordered_teams)/2)
top_teams = ordered_teams#[team_offset:team_offset+int(half_teams)]
print(f"top_teams {top_teams}")
excluded_teams = set(all_teams) - set(top_teams)
print(f"excluded_teams {excluded_teams}")

questionable_players = list(df[(~df['Injury Indicator'].isna()) | (~df['Injury Details'].isna())]['Name'])
low_salary_players = list(df[((df['Salary'] < MIN_SALARY)) & (df['Position'] != 'D')]['Name'])

excluded_players = [*questionable_players, *low_salary_players]
excluded_teams = list(df[(df['Position'] == 'D') & df['Team'].isin(excluded_teams)]['Name'])

if excluded_teams:
    excluded_players.extend(excluded_teams)

# excluded_players = ["Rob Gronkowski"]
print(f"Excluding: {len(excluded_players)}")

# custom_adds = ['Mike Evans', 'Tee Higgins', 'Los Angeles Rams', 'Tyler Higbee', 'Gabriel Davis', 'Sony Michel', 'Mecole Hardman']
# custom_adds = ['Byron Pringle', 'Tee Higgins', 'Gabriel Davis', 'Cedrick Wilson', 'Tyler Boyd', 'Tony Pollard','Mecole Hardman', 'Ezekiel Elliott', 'Tyler Higbee']
# custom_adds = ['Tyler Boyd', 'Stefon Diggs', 'Tyler Higbee']
# custom_adds = ['AJ Dillon', 'Tyler Boyd']
custom_adds = []

if excluded_players:
    excluded_players.extend(custom_adds)
    
questionable_df = df[(df['Name'].isin(set(questionable_players)))]
    
excluded_df = df[(df['Name'].isin(set(excluded_players)))]
    
df = df[(~df['Name'].isin(set(excluded_players)))]# & (df['FPPG'] > 0) & (df['Played'] > 0)

             FPPG      Played       Salary  Tier  Unnamed: 14  Unnamed: 15
count  129.000000  129.000000   129.000000   0.0          0.0          0.0
mean     4.292445    7.767442  4971.317829   NaN          NaN          NaN
std      5.039028    5.513494  1136.620113   NaN          NaN          NaN
min     -0.363636    1.000000  3400.000000   NaN          NaN          NaN
25%      0.600000    4.000000  4500.000000   NaN          NaN          NaN
50%      2.800000    6.000000  4500.000000   NaN          NaN          NaN
75%      5.360000   12.000000  5200.000000   NaN          NaN          NaN
max     23.037500   18.000000  9000.000000   NaN          NaN          NaN
ordered_teams [('GB', -5.45), ('TEN', -3.5), ('LAR', -3.47), ('KC', -2.48), ('BUF', 2.48), ('ARI', 3.47), ('CIN', 3.5), ('SF', 5.45)]
top_teams ['GB', 'TEN', 'LAR', 'KC', 'BUF', 'ARI', 'CIN', 'SF']
excluded_teams set()
Excluding: 116


In [62]:
print(winning_teams)

['LAR', 'TEN', 'GB', 'KC']


In [63]:
# http://rotoguru1.com/cgi-bin/fyday.pl?week=1&game=fd&scsv=1

start_week = WEEK-5# take last few games for momentum weighting.

file_names = [f"./history/week{week_number}.csv" for week_number in range(start_week, WEEK+1) if os.path.isfile(f"./history/week{week_number}.csv")]
print(file_names, len(file_names))

history_dfs = [pd.read_csv(f, delimiter=";") for f in file_names]
print(f"Using {len(history_dfs)} weeks of history")
historic_data=pd.concat(history_dfs)

if len(historic_data) > 0:
    REPLACE_MAP = {
        'LA': 'Los Angeles',
        '.':'',
    }

    def name_map(x):
        result = ' '.join(x.split(', ')[::-1])
        for k in REPLACE_MAP:
            result = result.replace(k, REPLACE_MAP[k])
        return result


    historic_data['Name'] = historic_data['Name'].apply(name_map)
    team_data = historic_data[historic_data['Pos'] == 'Def']

    historic_data[:1]
    historic_averages = historic_data.groupby("Name").mean()['FD points'].to_dict()
    historic_averages['Patrick Mahomes'] = historic_averages['Patrick Mahomes II']
    historic_averages['Darrell Henderson Jr'] = historic_averages['Darrell Henderson']
    # historic_averages

    team_averages = team_data.groupby("Team").mean()['FD points'].to_dict()
    team_averages['gb'] = team_averages.get('gnb')
    team_averages['kc'] = team_averages.get('kan')
    team_averages['ne'] = team_averages.get('nwe')
    team_averages['tb'] = team_averages.get('tam')
    team_averages['lv'] = team_averages.get('lvr')
    team_averages['no'] = team_averages.get('nor')
    team_averages['sf'] = team_averages.get('sfo')
    # print(team_averages)
    
    for k,v in team_averages.items():
        historic_averages[k] = team_averages[k]
print(len(historic_averages))

['./history/week15.csv', './history/week16.csv', './history/week17.csv', './history/week18.csv'] 4
Using 4 weeks of history
642


In [64]:
excluded_bonus = defaultdict(lambda: 0)

for index, p in questionable_df.iterrows():
    pos = p['Position']
    if pos in ['TE', 'WR', 'RB', 'QB']:
        points = p['FPPG']
        if points > 7:
            if pos == 'QB':
                # subtract for QB
                amt = -min(points * .08, .8)
            else:
                amt = min(points * .05, .6)
            print('bonus', p['Team'], p['Name'], amt)
            
            excluded_bonus[p['Team']] += amt
        
print(excluded_bonus)

bonus TEN Derrick Henry 0.6
bonus GB Aaron Jones 0.6
bonus SF Jimmy Garoppolo -0.8
bonus GB Randall Cobb 0.3618181748823686
defaultdict(<function <lambda> at 0x1530690e0>, {'TEN': 0.6, 'GB': 0.9618181748823686, 'SF': -0.8})


In [65]:
df.to_csv(ACTIVE_FILE)
print(f"wrote {ACTIVE_FILE}, {df.shape}")

wrote ./data/active.csv, (37, 19)


In [66]:
# https://www.fanduel.com/nfl-guide
# Use min/max + roster_size to accomodate flex position. FD roster_size = 9
def get_nfl_positions():
    return [
        ['QB', 1, 1],
        ['RB', 2 ,3],
        ['WR', 3, 4],
        ['TE', 1, 2],
        ['D', 1, 1]
    ]               

In [67]:
m_score = df.groupby(['Position'])['FPPG'].mean().to_dict()
m_score['DEF'] = m_score['D']
    
m_score

{'D': 6.919117647058824,
 'QB': 6.773270999998146,
 'RB': 10.620223628224297,
 'TE': 6.555441157023112,
 'WR': 10.93046897857846,
 'DEF': 6.919117647058824}

In [68]:
max_salary = np.percentile(df[df['Salary'] >= MIN_SALARY]['Salary'], 99)-1
max_salary = 9500
print(max_salary)

9500


In [69]:
df[df['Position'] == 'D'].shape

(4, 19)

In [70]:
ACTIVE_RULE_SET = rules.FD_NFL_RULE_SET
# Overrides (position limits, salary, roster size, positions, etc.
ACTIVE_RULE_SET.salary_max = 60000
ACTIVE_RULE_SET.salary_min = ACTIVE_RULE_SET.salary_max - 0
ACTIVE_RULE_SET.defensive_positions = ['D', 'DEF']
ACTIVE_RULE_SET.offensive_positions = ['QB', 'RB', 'WR', 'TE', 'FLEX', 'WR/FLEX']
ACTIVE_RULE_SET.position_limits = get_nfl_positions()
ACTIVE_RULE_SET.max_players_per_team = 9
ACTIVE_RULE_SET.roster_size = 9
print(ACTIVE_RULE_SET.__dict__)

ALL_POSITIONS = [*ACTIVE_RULE_SET.defensive_positions, *ACTIVE_RULE_SET.offensive_positions]

{'site': 'FAN_DUEL', 'league': 'NFL', 'roster_size': 9, 'position_limits': [['QB', 1, 1], ['RB', 2, 3], ['WR', 3, 4], ['TE', 1, 2], ['D', 1, 1]], 'general_position_limits': [], 'salary_min': 60000, 'salary_max': 60000, 'offensive_positions': ['QB', 'RB', 'WR', 'TE', 'FLEX', 'WR/FLEX'], 'defensive_positions': ['D', 'DEF'], 'game_type': 'classic', 'max_players_per_team': 9, 'position_per_team_rules': None, 'min_teams': None}


In [71]:
# Any additional player or custom rule constraints.
#Player -  https://github.com/BenBrostoff/draftfast/blob/68625902ceea83e66ee9f13a44acd732f600f68f/draftfast/orm.py#L245

# no players min cost (unlikely to play), low score, or favored to lose except overwhelming proj.
# Use salary data from csv as optimization basis.
# set((k, v['defensive_rating_rank']) for k,v in rankings.items())
# rankings['BUF']

allowed_map = get_fantasy_def_points_against(WEEK)
allowed_map

return cached data week 20


{'Jacksonville Jaguars': {'rank': 32, 'allowed': 9.71},
 'Atlanta Falcons': {'rank': 31, 'allowed': 9.59},
 'New York Giants': {'rank': 30, 'allowed': 9.59},
 'Carolina Panthers': {'rank': 29, 'allowed': 9.47},
 'Chicago Bears': {'rank': 28, 'allowed': 9.12},
 'Baltimore Ravens': {'rank': 27, 'allowed': 8.35},
 'New York Jets': {'rank': 26, 'allowed': 8.24},
 'Houston Texans': {'rank': 25, 'allowed': 7.65},
 'Washington Football Team': {'rank': 24, 'allowed': 7.53},
 'Denver Broncos': {'rank': 23, 'allowed': 7.53},
 'Miami Dolphins': {'rank': 22, 'allowed': 7.41},
 'Las Vegas Raiders': {'rank': 21, 'allowed': 6.94},
 'Detroit Lions': {'rank': 20, 'allowed': 6.65},
 'Cleveland Browns': {'rank': 19, 'allowed': 6.53},
 'New Orleans Saints': {'rank': 18, 'allowed': 6.47},
 'Tennessee Titans': {'rank': 17, 'allowed': 6.41},
 'Cincinnati Bengals': {'rank': 16, 'allowed': 6.24},
 'Pittsburgh Steelers': {'rank': 15, 'allowed': 6.06},
 'New England Patriots': {'rank': 14, 'allowed': 5.71},
 'Lo

In [72]:
players = salary_download.generate_players_from_csvs(salary_file_location=ACTIVE_FILE, game=rules.FAN_DUEL)

FAVOR_DIVISION = 10
AVERAGE_WEIGHT = .6
OVERUNDER_DIVISION = 4

defenses = []
qbs = []

historic_data_used = 0

for p in players:
    p.average_score = m_score[p.pos]
    name = p.name.replace('.', '')
    
    if WEIGHTED:
        base_score = p.proj
        
        history_key = name_map(p.name) if p.pos != 'D' else p.team.lower()
        history_value = historic_averages.get(history_key)
        if history_value:
            new_score = AVERAGE_WEIGHT*base_score + (1-AVERAGE_WEIGHT)*history_value
            base_score = new_score
            historic_data_used+=1
        else:
            new_score = None
            # print('no historic data', history_key)
            
        
        rank_bonus = -favor_map.get(p.team, 0)/FAVOR_DIVISION
        
        # print('rank_bonus', rank_bonus)
        teams = p.matchup.split('@')
        
        
        is_home = p.team == teams[1]
        if is_home:
            rank_bonus += 0.20

        teams.remove(p.team) # remove current players team
        opponent = teams[0]
        
        point_bonus = z_map.get(p.team, 0)
        if point_bonus:
            # max(0, point_bonus/4)
            overunder_bonus = -point_bonus/OVERUNDER_DIVISION if p.pos == 'D' else point_bonus/OVERUNDER_DIVISION # lower points better for defensive scoring.
            rank_bonus += overunder_bonus
            
        # current_rank = rankings[p.team]['overall_rating_rank']
        # opp_rank = rankings[opponent]['overall_rating_rank']
        current_rank = rankings[p.team]['points_rank']
        opp_rank = num_teams - rankings[opponent]['offensive_rating_rank']

        if p.pos == 'D':
            opp_bonus = excluded_bonus[opponent]/2
            rank_bonus += opp_bonus
            opp_def_avg = allowed_map[rankings[opponent]['team_fk__full_name']]['allowed']
            base_score = .5*base_score + .5*opp_def_avg
        else:
            if p.pos == 'QB':
                # negative (downside for QB injuries)
                injury_bonus = -excluded_bonus[p.team]
            else:
                injury_bonus = excluded_bonus[p.team] 
                
            rank_bonus += injury_bonus
            
        # overall_diff = opp_rank - current_rank # larger the better (should be between 1-32)
        # rank_bonus += overall_diff/num_teams*1.5
        p.proj = base_score + rank_bonus
        
        if p.pos == 'D':
            defenses.append((p.team, current_rank, opp_def_avg, opp_bonus, rank_bonus, base_score, p.proj, p.cost, p.proj / p.cost))
        elif p.pos == 'QB':
            qbs.append((history_key, history_value, base_score, overunder_bonus, p.proj, p.cost, p.proj / p.cost))
        
        if 'Jenn' in name or 'Uzomah' in name:# or p.pos == 'TE':
            print(history_key, history_value, base_score, overunder_bonus, p.proj, p.cost, p.proj / p.cost)
        

print(f"players {len(players)}")
print(f"historic data used {historic_data_used} of {len(players)}") 
# print(players)

for x in sorted(defenses, key=lambda x: x[-1], reverse=True):
    print(x)
# print(excluded_bonus)
print("\n---\n")
for x in sorted(qbs, key=lambda x: x[-1], reverse=True):
    print(x)

CJ Uzomah 4.866666666666666 6.1537253824869795 -0.19420873119061258 5.609516651296367 5500.0 0.0010199121184175212
Jauan Jennings 10.275 7.307142791748047 -0.1575140161515823 5.8046287755964645 5000.0 0.001160925755119293
players 37
historic data used 29 of 37
('CIN', 9, 6.41, 0.3, 0.1442087311906126, 6.238333333333333, 6.382542064523946, 3400.0, 0.0018772182542717488)
('TEN', 5, 6.24, 0.0, 0.7442087311906126, 6.6905882352941175, 7.43479696648473, 4100.0, 0.0018133651137767633)
('SF', 9, 3.53, 0.4809090874411843, 0.09342310359276662, 5.4816666666666665, 5.575089770259433, 3700.0, 0.001506781018989036)
('GB', 9, 5.18, -0.4, 0.5025140161515824, 5.472352941176471, 5.974866957328054, 4500.0, 0.0013277482127395674)

---

('Joe Burrow', 28.573333333333334, 23.63403878484987, -0.19420873119061258, 23.08983005365926, 7700.0, 0.0029986792277479557)
('Aaron Rodgers', 19.585, 20.46399954223633, -0.1575140161515823, 20.08966735120238, 8400.0, 0.002391627065619331)
('Trey Lance', 20.06, 14.71599981

In [73]:
player_settings = PlayerPoolSettings()
LOCKED = []
BLOCKED_TEAMS = []

# overrides:
# LOCKED = ['Cincinnati Bengals', 'Davante Adams']
# BLOCKED_TEAMS = []

constraints = LineupConstraints(locked=LOCKED)

MIN_PROJ = 5
min_favored = -8
MIN_LIMIT = -10

roster = None
best_roster = None
best_score = 0

get_score = lambda roster: sum([p.proj for p in roster.players])

while min_favored >= MIN_LIMIT:
    def block_function(p):
        store = p.kv_store
        played = int(float(store.get('Played', -1)))
        
        should_skip = ((p.team in BLOCKED_TEAMS or p.proj < MIN_PROJ) and p.pos != 'D') or (p.cost > max_salary) or played < MAX_PLAYED/2 or (p.pos == 'QB' and p.proj < 12)# or (favor_map.get(p.team, 0) > -min_favored)
    
        #print(p.name, played, MAX_PLAYED)
        return should_skip

    opt_settings = OptimizerSettings(
        custom_rules=[
            CustomRule(
                group_a=lambda p: p,
                group_b=block_function, 
                comparison=lambda sum, a, b: sum(b) == 0
            ),
        ],
        min_teams=2
    )

    roster = run(
        rule_set=ACTIVE_RULE_SET,
        player_pool=players,
        verbose=False,
        optimizer_settings=opt_settings,
        constraints=constraints,
        player_settings=player_settings
    )
    
    if roster:
        # total_salary = sum([p.cost for p in roster.players])
        print(roster)
        print('min_favored', min_favored)# negative means players' teams can lose
        print(f"spread weighted: {WEIGHTED}\n\n")
        current_score = get_score(roster)
        if not best_score or current_score > best_score:
            best_score = current_score
            best_roster = roster

    min_favored -= 1
    
    if min_favored >= MIN_LIMIT:
        roster = None
        
if not roster:
    print("No solution")
elif not favor_map:
    print("Warning: No favor map used")

+----------+------------------+------+---------+--------+--------------------+----------+--------+
| Position | Player           | Team | Matchup | Salary |         Projection | vs. Avg. | Locked |
+----------+------------------+------+---------+--------+--------------------+----------+--------+
| QB       | Joe Burrow       | CIN  | CIN@TEN |  7,700 |  23.08983005365926 |    16.32 |        |
| RB       | Joe Mixon        | CIN  | CIN@TEN |  7,800 | 15.494614798221152 |     4.87 |        |
| RB       | AJ Dillon        | GB   | SF@GB   |  6,100 | 12.059892609443104 |     1.44 |        |
| WR       | Davante Adams    | GB   | SF@GB   |  8,700 | 19.694303700967115 |     8.76 |        |
| WR       | Deebo Samuel     | SF   | SF@GB   |  8,600 | 18.070897446968825 |     7.14 |        |
| WR       | Allen Lazard     | GB   | SF@GB   |  6,000 | 12.279304158730787 |     1.35 |        |
| WR       | Tyler Boyd       | CIN  | CIN@TEN |  5,800 | 11.239948325430865 |     0.31 |        |
| TE      

In [74]:
# def find_id(first_name, last_name, pos):
#     matches = df.loc[(df['First Name'] == first_name) & (df['Last Name'] == last_name) & (df['Position'] == pos)]
#     if matches:||
#         return matches.iloc[0]['Id']
#     return None
#print(f"{"Player":20}{"Advantage":10}")
if favor_map:
    roster = best_roster
    sorted_players = sorted(roster.players, key=lambda x: favor_map[x.team])
    net_score = 0
    for p in sorted_players:
        advantage = favor_map[p.team]
        name = p.name.replace('.', '')
        print(f"{name:24}{advantage:>10}{historic_averages.get(name, ''):>20} {p.proj} {p.kv_store['Played']} {excluded_bonus[p.team]}")#, p.kv_store['Played'], MAX_PLAYED)
        net_score += advantage
    print(f"---\nTotal adv: {net_score}\n")
    roster.players[-1].__dict__



Davante Adams                -5.45               18.85 19.694303700967115 16.0 0.9618181748823686
AJ Dillon                    -5.45  11.400000000000002 12.059892609443104 17.0 0.9618181748823686
Allen Lazard                 -5.45                13.7 12.279304158730787 14.0 0.9618181748823686
Anthony Firkser               -3.5   6.550000000000001 6.059791207774232 15.0 0.6
Tennessee Titans              -3.5                     7.43479696648473 17.0 0.6
Joe Mixon                      3.5  15.700000000000001 15.494614798221152 17.0 0
Joe Burrow                     3.5  28.573333333333334 23.08983005365926 17.0 0
Tyler Boyd                     3.5  15.233333333333334 11.239948325430865 17.0 0
Deebo Samuel                  5.45               20.79 18.070897446968825 17.0 -0.8
---
Total adv: -7.400000000000001



In [75]:
for r in roster.players:
    print(r.team, r.name, r.proj, r.average_score, r.cost,  r.kv_store['Injury Indicator'], r.kv_store['Injury Details'], r.kv_store['Played'])
    # if 'Zay' in r.name:
    #     print(r.__dict__)

GB Davante Adams 19.694303700967115 10.93046897857846 8700.0   16.0
SF Deebo Samuel 18.070897446968825 10.93046897857846 8600.0   17.0
CIN Joe Mixon 15.494614798221152 10.620223628224297 7800.0   17.0
CIN Joe Burrow 23.08983005365926 6.773270999998146 7700.0   17.0
GB AJ Dillon 12.059892609443104 10.620223628224297 6100.0   17.0
GB Allen Lazard 12.279304158730787 10.93046897857846 6000.0   14.0
CIN Tyler Boyd 11.239948325430865 10.93046897857846 5800.0   17.0
TEN Anthony Firkser 6.059791207774232 6.555441157023112 5200.0   15.0
TEN Tennessee Titans 7.43479696648473 6.919117647058824 4100.0   17.0


In [76]:
# Create upload CSV by reordering columns to match template order.

headers = []
players = []
ORDERED_COLS = ['QB','RB','RB','WR','WR','WR','TE','FLEX','DEF'] # template order.

def get_match_names(col):
    if col == 'DEF':
        return ['D']
    elif col == 'FLEX' :
        return ['RB', 'WR']
    return [col]

roster_copy = roster.players.copy()
for c in ORDERED_COLS:
    headers.append(c)
    match_names = get_match_names(c)
    for r in roster_copy:
        if r.pos in match_names:
            p = f"{r.kv_store['Id']}:{r.name}"
            players.append(p)
            roster_copy.remove(r)
            break

with open('upload.csv', 'w') as f:
    f.write(','.join(headers))
    f.write('\n')
    f.write(','.join(players))

print('done')


done


In [77]:
output = pd.read_csv('upload.csv')
output

,QB,RB,RB.1,WR,WR.1,WR.2,TE,FLEX,DEF
0,70436-63336:Joe Burrow,70436-52442:Joe Mixon,70436-93107:AJ Dillon,70436-45889:Davante Adams,70436-56018:Deebo Samuel,70436-56809:Allen Lazard,70436-80294:Anthony Firkser,70436-40669:Tyler Boyd,70436-12534:Tennessee Titans


In [78]:
m_score = df[(df['FPPG'] > 0) & (df['Played'] > 0)].groupby(['Position'])['FPPG'].mean().to_dict()
m_score['DEF'] = m_score['D']
m_score['FLEX'] = (m_score['WR']+m_score['RB'])/2
m_score

{'D': 6.919117647058824,
 'QB': 9.087694666432398,
 'RB': 10.620223628224297,
 'TE': 6.555441157023112,
 'WR': 10.93046897857846,
 'DEF': 6.919117647058824,
 'FLEX': 10.775346303401378}

In [79]:
expected_score = sum([m_score.get(h) for h in headers])
expected_score

87.36945396609968

In [80]:
# Players that have non-null injury status.
df[(~df['Injury Indicator'].isna()) | ~df['Injury Details'].isna()]

,Id,Position,First Name,Nickname,Last Name,FPPG,Played,Salary,Game,Team,Opponent,Injury Indicator,Injury Details,Tier,Unnamed: 14,Unnamed: 15,Roster Position,Name,Salary/FPPG


In [81]:
df[df['First Name'] == 'Zay']

,Id,Position,First Name,Nickname,Last Name,FPPG,Played,Salary,Game,Team,Opponent,Injury Indicator,Injury Details,Tier,Unnamed: 14,Unnamed: 15,Roster Position,Name,Salary/FPPG


In [82]:
# Potentially unaccounted positions
df[~df['Position'].isin(ALL_POSITIONS)]

,Id,Position,First Name,Nickname,Last Name,FPPG,Played,Salary,Game,Team,Opponent,Injury Indicator,Injury Details,Tier,Unnamed: 14,Unnamed: 15,Roster Position,Name,Salary/FPPG


In [83]:
df[df['Position'] == 'D'][['FPPG', 'Nickname', 'Salary', 'Salary/FPPG']].sort_values('Salary/FPPG', ascending=False)

,FPPG,Nickname,Salary,Salary/FPPG
127,7.055556,San Francisco 49ers,3700,0.001907
128,6.444444,Cincinnati Bengals,3400,0.001895
109,7.235294,Tennessee Titans,4100,0.001765
99,6.941176,Green Bay Packers,4500,0.001542


In [84]:
df[df['Team'] == 'JAC']

,Id,Position,First Name,Nickname,Last Name,FPPG,Played,Salary,Game,Team,Opponent,Injury Indicator,Injury Details,Tier,Unnamed: 14,Unnamed: 15,Roster Position,Name,Salary/FPPG


In [85]:
spread_df.iloc[1][['HomeTeam', 'PointSpread']]

HomeTeam       TEN
PointSpread   -3.5
Name: 1, dtype: object

In [86]:
from pydfs_lineup_optimizer import Site, Sport, get_optimizer

optimizer = get_optimizer(Site.FANDUEL, Sport.FOOTBALL)
optimizer.load_players_from_csv(ACTIVE_FILE)
for lineup in optimizer.optimize(10):
    print(lineup)


 1. QB      Joe Burrow                    QB    CIN            CIN@TEN  20.341         7700.0$   
 2. RB      D'Onta Foreman                RB    TEN            CIN@TEN  9.933          5500.0$   
 3. RB      Dontrell Hilliard             RB    TEN            CIN@TEN  9.029          5100.0$   
 4. WR      Davante Adams                 WR    GB             SF@GB    17.675         8700.0$   
 5. WR      Deebo Samuel                  WR    SF             SF@GB    18.762         8600.0$   
 6. WR      Tee Higgins                   WR    CIN            CIN@TEN  12.24          6400.0$   
 7. TE      George Kittle                 TE    SF             SF@GB    10.987         6400.0$   
 8. FLEX    Joe Mixon                     RB    CIN            CIN@TEN  16.265         7800.0$   
 9. DEF     San Francisco 49ers           D     SF             SF@GB    7.056          3700.0$   

Fantasy Points 122.29
Salary 59900.00

 1. QB      Joe Burrow                    QB    CIN            CIN@TEN  20.341